In [ ]:
from dotenv import load_dotenv
import os
import pandas as pd
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain.chains import RetrievalQA
from langchain_community.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

In [ ]:
# RAG con LangChain + Zephyr + Chroma para datos de rendimiento físico

## Instalación de dependencias (sólo una vez)
# !pip install pandas langchain langchain-community chromadb sentence-transformers transformers accelerate python-dotenv



# 1. Cargar variables de entorno (opcional si usas otros recursos privados)
load_dotenv()

# 2. Leer CSV de datos
df = pd.read_csv("data/processed/merged_df.csv")

# 3. Convertir cada fila del DataFrame en un documento para embeddings
docs = [Document(page_content="\n".join([f"{col}: {val}" for col, val in row.items()])) for _, row in df.iterrows()]

# 4. Dividir los documentos en chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
split_docs = splitter.split_documents(docs)

# 5. Embeddings usando modelo local (MiniLM)
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# 6. Crear y guardar vector store en Chroma
vectordb = Chroma.from_documents(split_docs, embedding=embedding_model, persist_directory="./chroma_index")
vectordb.persist()

# 7. Cargar modelo instruct gratuito: Zephyr 7B Alpha
model_name = "HuggingFaceH4/zephyr-7b-alpha"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype="auto")

# 8. Crear pipeline de generación
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.7,
    top_k=50,
    top_p=0.95,
)

llm = HuggingFacePipeline(pipeline=pipe)

# 9. Crear el RAG QA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True
)

# 10. Consulta de ejemplo
query = "¿Cuál fue el día con mayor distancia recorrida y cómo estaba el estado de recuperación?"
response = qa_chain.invoke(query)

print("\n📣 Respuesta generada:\n")
print(response["result"])
